## Algoritmos de otimização

Descrição: A plataforma smarttbot é uma empresa de tecnologia especializada em investimento automatizado na bolsa de valores. Ela possibilita que investidores escolham estratégias que são formadas por indicadores  e configurem parâmetros dos principais indicadores do mercado, existem diversas estratégias, mas o escopo desse projeto vai considerar a tangram. Os parâmetros  
Objetivo: Encontrar os melhores parâmetros da estratégia que retornem um maior lucro.

In [225]:
#importação das bibliotecas
import time
import random
import math
import pandas as pd
import numpy as np
import chardet

In [226]:
#¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨
#Serão constantes
ticket = 'WIN'                                     #papel negociado
direction_operations = 'comprado&vendido'          #operações possiveis: compra e venda a descoberto
amount_orders = 1                                  #quantidade de ordens 
start_time = '09:00'                               #horário inicial
end_time = '17:00'                                 #horário limite de operação
#¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨
#Serão variáveis
time_frame = 1                                     #tempo gráfico em min
stop = -200                                        #limite de perda por operação
gain = 200                                         #algo de ganho por operação

In [227]:
# carregando os dados
with open('M4.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

base = pd.read_csv('M4.csv', encoding=result['encoding'],dtype='unicode')

colunas = ['Data','Hora', 'open', 'high', 'low', 'close','VOL']
base = base[colunas][1:200]
base.head()

,Data,Hora,open,high,low,close,VOL
1,2014.10.20,09:15,56265.00,56270.00,56250.00,56260.00,65.00
2,2014.10.20,09:16,56260.00,56290.00,56250.00,56290.00,80.00
3,2014.10.20,09:17,56285.00,56300.00,56270.00,56270.00,124.00
4,2014.10.20,09:18,56280.00,56290.00,56270.00,56285.00,25.00
5,2014.10.20,09:19,56290.00,56375.00,56290.00,56335.00,237.00


In [228]:
#Calculando alguns dos indicadores para alteração de parâmetros
class Indicators():
    def __init__(self,name='indicators'):
        self.name = name    
    def sma(self,data,period,column='close'):
        sma_ = pd.Series(data[column].rolling(period).mean(),name ='SMA'+str(period))
        return data.join(sma_)
    def ema(self,data,period,column='close'):
        data['EMA'+str(period)]=data[column].ewm(ignore_na=False,min_periods=period,com=period,adjust=True).mean()
        return data
    def macd(self,data,period_long=26,period_short=12,period_signal=9,column='close'):
        remove_cols = []
        if not 'EMA'+str(period_long) in data.columns:
            data = self.ema(data,period_long)
            remove_cols.append('EMA'+str(period_long))
            
        if not 'EMA'+str(period_short) in data.columns:
            data = self.ema(data, period_short)
            remove_cols.append('EMA' + str(period_short))
    
        data['macd_val'] = data['EMA' + str(period_short)] - data['EMA' + str(period_long)]
        data['macd_signal_line'] = data['macd_val'].ewm(ignore_na=False, min_periods=0, com=period_signal, adjust=True).mean()
        data = data.drop(remove_cols, axis=1)
        return data
    def bands(self,data, trend_periods=20, deviation=2, column='close'):
        data['bol_bands_middle'] = data[column].ewm(ignore_na=False, min_periods=0, com=trend_periods, adjust=True).mean()
        for index, row in data.iterrows():
            s = data[column].iloc[index - trend_periods: index]
            sums = 0
            middle_band = data.at[index, 'bol_bands_middle']
            for e in s:
                #e = float(e)
                #print('verificacao:',e,middle_band,type(e),type(middle_band))
                #print('----------------------')
                sums += np.square(e - middle_band)
            
            std = np.sqrt(sums / trend_periods)
            
            upper_band = middle_band + (deviation * std)
            lower_band = middle_band - (deviation * std)
            
            data.at[index,'bol_bands_upper']= upper_band
            data.at[index,'bol_bands_lower']= lower_band
        return data
    def ifr(self,data,period=14):
        delta = data['close'].diff().dropna()
        u = delta * 0
        d = u.copy()
        u[delta > 0] = delta[delta > 0]
        d[delta < 0] = -delta[delta < 0]
        u = u.ewm(span=period,min_periods=0,adjust=False).mean() #first value is sum of avg gains
        d = d.ewm(span=period,min_periods=0,adjust=False).mean() #first value is sum of avg losses
        data['IFR']=1 - 1 / (1 + u/d)
        return data
    def vwap(self,data):
        vol = data.VOL.values
        price = data.close.values
        data['VWAP'] = (vol*price).cumsum()/vol.cumsum()
        return data
        

In [229]:
# Instância do objeto Indicators, calculando os valores dos indicadores
indicators = Indicators()
base1 = base[['open', 'high', 'low', 'close','VOL']].astype(float)
base1 = indicators.sma(base1,3)
base1 = indicators.ema(base1,6)
base1 = indicators.macd(base1,26,12,9)
base1 = indicators.bands(base1,20,2)
base1 = indicators.ifr(base1,14)
base1 = indicators.vwap(base1)
base1 = base1.dropna()
base1.head(10)


,open,high,low,close,VOL,SMA3,EMA6,macd_val,macd_signal_line,bol_bands_middle,bol_bands_upper,bol_bands_lower,IFR,VWAP
26,56080.0,56100.0,56015.0,56020.0,265.0,56065.000000,56126.225254,-24.413831,-24.413831,56176.127521,56374.521030,55977.734013,0.278147,56182.510740
27,56020.0,56060.0,56020.0,56050.0,98.0,56046.666667,56115.163644,-26.140895,-25.322812,56167.924204,56357.750978,55978.097431,0.357465,56179.308261
28,56050.0,56060.0,56030.0,56060.0,60.0,56043.333333,56107.176493,-27.264924,-26.039458,56161.025018,56332.402683,55989.647353,0.383518,56177.568651
29,56055.0,56085.0,56045.0,56055.0,88.0,56055.000000,56099.636427,-28.304578,-26.698115,56154.355992,56299.858190,56008.853794,0.374752,56175.002379
30,56050.0,56060.0,56035.0,56050.0,27.0,56055.000000,56092.475269,-29.271206,-27.326449,56147.890747,56284.585017,56011.196478,0.365122,56174.204492
31,56035.0,56035.0,55995.0,56005.0,306.0,56036.666667,56079.872852,-31.384015,-28.192416,56139.163236,56268.836914,56009.489559,0.288211,56162.789903
32,56005.0,56010.0,55955.0,55990.0,155.0,56015.000000,56066.940682,-33.592627,-29.227528,56130.173606,56261.441651,55998.905561,0.266611,56157.080580
33,55985.0,55995.0,55930.0,55940.0,109.0,55978.333333,56048.693592,-36.964306,-30.585971,56118.855552,56263.858365,55973.852739,0.206956,56152.151042
34,55930.0,56020.0,55930.0,55990.0,262.0,55973.333333,56040.264165,-38.252765,-31.837530,56111.276950,56258.495907,55964.057992,0.369688,56143.758396
35,56005.0,56040.0,55980.0,55995.0,148.0,55975.000000,56033.768378,-39.109452,-32.954017,56104.513872,56251.009402,55958.018342,0.384267,56139.532630


In [230]:
# Classe trade, faz as operações e retorna o ganho, assim como dataframe com todas as operações realizadas
class Env_trade():
    comprado = False
    vendido = False
    ficha = 0
    cont = 0
    valor = 0
    contC = 0
    contV = 0
    hora = 0
    Stop = 100
    posicao = 0

    def __init__(self,name='wind'):
        self.name = name
        self.CC = []
        self.VV = []
        self.saida = pd.DataFrame(columns=['tipo','entrada','saida','ganho','inicio','fim','duracao','ganhofinal','acumulado','recompensas'])
        self.rewards = []
        self.atual = 0
        self.result = 0
        self.rec = 0
        
        
    def agente_(self,dados,acao,stop,gain,verbose,forma):
        self.cont += 1    
        self.posicao = 0
        self.result = 0
        
        hora = float(dados[0].split(':')[0])
        if hora >= 17 and self.comprado == False and self.vendido == False:
            return self.CC,self.VV, self.saida,self.ficha,self.comprado,self.vendido,0
        if acao == 0:
            recompensa = 0
        elif acao == 1 and self.comprado == False:
            if self.ficha == 0:
                self.ficha = 1
                self.comprado = True
                self.vendido = False
            else:
                self.ficha = 0
                self.comprado = False
                self.vendido = False
            self.posicao = self.compra(dados[1],dados[0],self.ficha,verbose)
            
        elif acao == 2 and self.vendido == False: 
            if self.ficha == 0:
                self.ficha = 1
                self.vendido = True
                self.comprado = False
            else:
                self.ficha = 0
                self.vendido = False
                self.comprado = False
            self.posicao = self.venda(dados[1],dados[0],self.ficha,verbose)
            
        if self.ficha == 1:
            rec = self.valor -dados[4]
            if self.vendido:
                self.rewards.append(rec)
            else:
                self.rewards.append(-rec)
        if self.comprado or self.vendido:
            self.posicao = self.verifica(dados,gain,stop,verbose)

        recompensa = 0
        if self.rec != 0:
            recompensa = self.rec
            self.rec = 0
            
        
        if forma == 0:
            if self.comprado:
                recompensa = dados[4] - self.valor 
            if self.vendido:
                recompensa = self.valor - dados[4]
        if forma == 1:
            if self.comprado == False and self.vendido == False:
                self.atual = 0
            if self.comprado:
                recompensa = dados[4] - self.valor
                if self.atual == 0:
                    self.atual = recompensa
                else:
                    dif = recompensa - self.atual
                    self.atual = recompensa
                    recompensa = dif
            if self.vendido:
                recompensa = self.valor - dados[4]
                if self.atual == 0:
                    self.atual = recompensa
                else:
                    dif = recompensa - self.atual
                    self.atual = recompensa
                    recompensa = dif
        if forma == 2:
            recompensa = 0
            if self.posicao == 4 or self.posicao == 2:
                recompensa = self.result
        if verbose == 1:
            print('##########################')
            print('Contador: ',self.cont)
            # print('Posicao: ',self.posicao)
            print('Hora: ',dados[0])
            print('Open: ',dados[1])
            print('High: ',dados[2])
            print('Low: ',dados[3])
            print('close: ',dados[4])
            print('acao: ',acao)
            print('stop: ',stop)
            print('gain: ',gain)
            print('Comprado: ',self.comprado)
            print('Vendido: ',self.vendido)
            # print('Ficha: ',self.ficha)
            # print('Compras: ',self.CC)
            # print('Vendas: ',self.VV)
            # print('Qtd: ',self.contC,self.contV)
            print('recompensa: ',recompensa)
            print('##########################')
                
        return self.saida,self.comprado,self.vendido,recompensa
                
    def verifica(self,dados,gain,stop,verbose):
        if self.comprado:
            stopmax = dados[3]-self.valor
            gainmin = dados[2]-self.valor

            if stopmax < stop:
                pontos = self.valor - (-stop)
                self.ficha = 0
                self.vendido = False
                self.comprado = False
                self.rec = stop
                return self.venda(pontos,dados[0],self.ficha,verbose)
                
            if gainmin > gain: 
                pontos = self.valor + gain
                self.ficha = 0
                self.vendido = False
                self.comprado = False
                self.rec = gain
                return self.venda(pontos,dados[0],self.ficha,verbose)
                
            
        if self.vendido:
            stopmax = self.valor - dados[2]
            gainmin = self.valor - dados[3]
            if stopmax < stop:
                pontos = self.valor + (-stop)
                self.ficha = 0
                self.vendido = False
                self.comprado = False
                self.rec = stop
                return self.compra(pontos,dados[0],self.ficha,verbose)
            if gainmin > gain:
                pontos = self.valor - gain
                self.ficha = 0
                self.vendido = False
                self.comprado = False
                self.rec = gain
                return self.compra(pontos,dados[0],self.ficha,verbose)
        return self.posicao

    def imprimi(self,pmax,pmin):
        print('---------------------------')
        print('valor da compra: ',self.valor)
        print('pontuacao máxima: ',pmax)
        print('pontuacao mínima: ',pmin)
        print('---------------------------')
    def recompensa(self,dados):
        return dados
    def compra(self,entrada,hora,ficha,verbose):
        if ficha == 1:
            self.valor = entrada
            self.contC += 1 
            self.hora = hora
            
            if verbose == 1:
                print('        ')
                print('********************')
                print('COMPRA: ',entrada)
                print('********************')
                print('        ')
            return 1
        else:
            if verbose == 1:
                print('        ')
                print('********************')
                print('VENDIDO: ',self.valor)
                print('COMPRA: ',entrada)
                print('Lucro: ',self.valor - entrada)
                print('inicio: ',self.hora,' termino: ',hora)
                print('Duracao: ',self.Duracao(self.hora,hora))
                print('recompensas: ',self.rewards)
                print('********************')
                print('        ')
            self.result = self.valor - entrada
            tempo = self.Duracao(self.hora,hora)
            self.VV.append(self.result)
            self.ficha = 0
            self.saida = self.saida.append({'tipo': 'venda',
                                            'entrada':self.valor,
                                            'saida':entrada,
                                            'ganho': self.result,
                                            'inicio':self.hora,
                                            'fim':hora,
                                            'duracao': tempo,
                                            'ganhofinal': self.result - tempo,
                                            'acumulado': sum(self.saida.ganhofinal),
                                            'recompensas': self.rewards}, ignore_index=True)
            self.rewards = []
            return 2
        # return 1
        
    def venda(self,entrada,hora,ficha,verbose):
        if ficha == 1:
            self.valor = entrada
            self.contV += 1
            self.hora = hora
            
            if verbose == 1:
                print('        ')
                print('********************')
                print('VENDA: ',entrada)
                print('********************')
                print('        ')
            return 3
        else:
            if verbose == 1:
                print('        ')
                print('********************')
                print('COMPRADO: ',self.valor)
                print('VENDA: ',entrada)
                print('LUCRO: ',entrada - self.valor)
                print('inicio: ',self.hora,' termino: ',hora)
                print('Duracao: ',self.Duracao(self.hora,hora))
                print('recompensas: ',self.rewards)
                print('********************')
                print('        ')
            self.result = entrada - self.valor
            tempo = self.Duracao(self.hora,hora)
            self.CC.append(self.result )
            self.ficha = 0
            self.saida = self.saida.append({'tipo': 'compra',
                                            'entrada':self.valor,
                                            'saida':entrada,
                                            'ganho': self.result,
                                            'inicio':self.hora,
                                            'fim':hora,
                                            'duracao': tempo,
                                            'ganhofinal': self.result - tempo,
                                            'acumulado': sum(self.saida.ganhofinal),
                                            'recompensas': self.rewards}, ignore_index=True)
            self.rewards = []
            return 4          
        
    def Duracao(self,base1a,base2a):
        base1 = float(base1a.split(':')[1])
        base2 = float(base2a.split(':')[1])
        base3 = float(base1a.split(':')[0])
        base4 = float(base2a.split(':')[0])
        if base1 > base2:
            tempo = ((60*(base4-base3)) - base1)+ base2
        else:
            tempo = base2 - base1
        if base3 > 17 or base4 > 17 or tempo > 50:
            tempo = 0
        # print((base4-base3),base1,base2,tempo)
        return tempo
    
    def reset(self):
        self.saida = self.saida.drop([i for i in range(len(self.saida))])
        self.ficha = 0
        self.cont = 0
        self.VV = []
        self.CC = []
        self.rewards = []
        self.comprado = False
        self.vendido = False

    

In [231]:
trader = Env_trade()
base = base.drop(['Data'],axis=1)
base['open'] = base['open'].astype(float)
base['high'] = base['high'].astype(float)
base['low'] = base['low'].astype(float)
base['close'] = base['close'].astype(float)
#agente(self,dados,acao,stop,gain,verbose=0,forma=0):
for i in range(100):
    action = random.randrange(0,3)
    print('action: ',action)
    result,buy,shell,reward = trader.agente_(base.values[i],action,stop,gain,1,0)

action:  1
        
********************
COMPRA:  56265.0
********************
        
##########################
Contador:  1
Hora:  09:15
Open:  56265.0
High:  56270.0
Low:  56250.0
close:  56260.0
acao:  1
stop:  -200
gain:  200
Comprado:  True
Vendido:  False
recompensa:  -5.0
##########################
action:  0
##########################
Contador:  2
Hora:  09:16
Open:  56260.0
High:  56290.0
Low:  56250.0
close:  56290.0
acao:  0
stop:  -200
gain:  200
Comprado:  True
Vendido:  False
recompensa:  25.0
##########################
action:  2
        
********************
COMPRADO:  56265.0
VENDA:  56285.0
LUCRO:  20.0
inicio:  09:15  termino:  09:17
Duracao:  2.0
recompensas:  [-5.0, 25.0]
********************
        
##########################
Contador:  3
Hora:  09:17
Open:  56285.0
High:  56300.0
Low:  56270.0
close:  56270.0
acao:  2
stop:  -200
gain:  200
Comprado:  False
Vendido:  False
recompensa:  0
##########################
action:  2
        
********************
VEND

action:  0
##########################
Contador:  73
Hora:  10:27
Open:  56100.0
High:  56180.0
Low:  56085.0
close:  56145.0
acao:  0
stop:  -200
gain:  200
Comprado:  True
Vendido:  False
recompensa:  80.0
##########################
action:  1
##########################
Contador:  74
Hora:  10:28
Open:  56140.0
High:  56240.0
Low:  56130.0
close:  56235.0
acao:  1
stop:  -200
gain:  200
Comprado:  True
Vendido:  False
recompensa:  170.0
##########################
action:  2
        
********************
COMPRADO:  56065.0
VENDA:  56240.0
LUCRO:  175.0
inicio:  10:24  termino:  10:29
Duracao:  5.0
recompensas:  [-35.0, -35.0, 45.0, 80.0, 170.0]
********************
        
##########################
Contador:  75
Hora:  10:29
Open:  56240.0
High:  56295.0
Low:  56200.0
close:  56285.0
acao:  2
stop:  -200
gain:  200
Comprado:  False
Vendido:  False
recompensa:  0
##########################
action:  0
##########################
Contador:  76
Hora:  10:30
Open:  56275.0
High:  56290.0
L

In [232]:
gain

200